In [ ]:
#!/usr/bin/python

from RandomAccessBag import ImageBag
import numpy as np
from place_recognizer.srv import place_recognizer
from cv_bridge import CvBridge, CvBridgeError
import rospy
import pickle

bridge = CvBridge()

def runPlaceRecognizerImg(imageArray):
    msg = bridge.cv2_to_imgmsg(imageArray, "bgr8")
    server = rospy.ServiceProxy('place_recognizer', place_recognizer)
    placeResp = server(msg)
    return placeResp.keyframeId

queries=np.load('/media/sujiwo/PlaceRecognition/queries.npy')
queryBag=ImageBag('/media/sujiwo/PlaceRecognition/ouster64-prep-4.bag', '/front_rgb/image_raw')

queryResults = {}
for q in queries:
    image = queryBag[q]
    qres = runPlaceRecognizerImg(image)
    queryResults[q] = qres
    print("--- Q {}: {}".format(q, len(qres)))
print("Queries done for {} images".format(len(queries)))

pickle.dump(queryResults, open("/media/sujiwo/PlaceRecognition/queryResults.pickle", "wb"))


In [ ]:
# Evaluation
queryResults=pickle.load(open("/media/sujiwo/PlaceRecognition/queryResults.pickle", "rb"))
queryPos=np.loadtxt("/media/sujiwo/PlaceRecognition/ouster64-conv.image.csv")
trainPos=np.loadtxt('/media/sujiwo/PlaceRecognition/vls128-conv.image.csv')

# Modify this
acceptDistLimit = 20.0

evalResults = {}
for q in queryResults.keys():
    qpos = queryPos[q, 1:4]
    for ans in queryResults[q]:
        evalResults[qpos]=
        atpos = trainPos[ans, 1:4]
        d = np.linalg.norm(atpos - qpos)
        if (d < acceptDistLimit):
            evalResults[qpos][ans]=1

